In [1]:
import dm_event
import time

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.optimize import minimize


from scipy.interpolate import CubicSpline

plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams.update({'font.size': 20})

Welcome to JupyROOT 6.22/07


# Upper limit SI

In [ ]:

nx, ny = 4000, 4000
mass_ccd = 6e-3
masked_fraction = (2+0.25)/100
nccd = 1
total_mass = nccd*mass_ccd*masked_fraction


dc = (15*15)*1e-6
print("Dark Current [e/pix_binned*days]: ", dc)#*1e8/(15**2*6.28*1e18*24*3600))
tread = 8.3*100/(30*24)
t_exp = 38*tread
print("Total Exposure [days]: ", mass_ccd*1e3*t_exp)
xmin, xmax = -10,10
xs_35 = []
mass_x = np.geomspace(0.6, 100, 10)*1e6
noise = 1.6

#df_inf = pd.read_csv("DAMICM_Upper.csv", comment="#")
#mass_x = df_inf["mx"]

nPeaks = 3
n_reps = 1
xs_0 = np.zeros([len(mass_x),n_reps])
xs_1 = np.zeros([len(mass_x),n_reps])
xs_2 = np.zeros([len(mass_x),n_reps])

for i,m in enumerate(mass_x):
    for j in range(n_reps):
        print("------Mass {} GeV-------".format(m))
        dm = dm_event.dm_event(m,1e-33,0,total_mass,t_exp,noise, xmin=xmin, xmax=xmax,ccd_mass=mass_ccd,
                              tread=tread, nccd=nccd,nx=nx,ny=ny)

        dm.verbose()
        dm.likelihood(x0=[dm.npix,0,noise,1,dc,-33],fix_pars=[1,2,3,4],
                      pars_lims=[(1e4,1e10),(-0.1,0.1),(0.07,2.5),(0.8,1.2),(1e-4,1e-2),(-36,-25)], 
                      verbose=False, simulate=dc, upper_limit=[1.355,True], nPeaks=nPeaks, bin_size=0.01)
        #print(dm.cross_section_dLL)
        xs_0[i][j] = dm.cross_section_dLL
        
        
        dm = dm_event.dm_event(m,1e-33,1,total_mass,t_exp,noise, xmin=xmin, xmax=xmax,ccd_mass=mass_ccd,
                              tread=tread, nccd=nccd,nx=nx,ny=ny)

        dm.verbose()
        dm.likelihood(x0=[dm.npix,0,noise,1,dc,-33],fix_pars=[0,1,2,3,4],
                      pars_lims=[(1e4,1e10),(-0.1,0.1),(0.07,2.5),(0.8,1.2),(1e-4,1e-2),(-36,-25)], 
                      verbose=False, simulate=dc, upper_limit=[1.355,True], nPeaks=nPeaks)
        #print(dm.cross_section_dLL)
        xs_1[i][j] = dm.cross_section_dLL
        
        
        dm = dm_event.dm_event(m,1e-33,2,total_mass,t_exp,noise, xmin=xmin, xmax=xmax,ccd_mass=mass_ccd,
                              tread=tread, nccd=nccd,nx=nx,ny=ny)

        dm.verbose()
        dm.likelihood(x0=[dm.npix,0,noise,1,dc,-33],fix_pars=[0,1,2,3,4],
                      pars_lims=[(1e4,1e10),(-0.1,0.1),(0.07,2.5),(0.8,1.2),(1e-4,1e-2),(-36,-25)], 
                      verbose=False, simulate=dc, upper_limit=[1.355,True], nPeaks=nPeaks)
        #print(dm.cross_section_dLL)
        xs_2[i][j] = dm.cross_section_dLL

        """
        plt.hist(dm.events, bins = 10, density = False)
        dm = dm_event.dm_event(14, 14, m, dm.cross_section_95, 0.6, 1, True, Eemin=0.06, Eemax=Eemax, eff="efficiency_DAMIC.csv")
        dm.plot_var("dRdE")
        plt.loglog()
        plt.xlim(0.06,1)
        plt.ylim(1e-3,1e2)
        plt.show()
        """
        
    #df_inf = pd.read_csv("DAMICM_Upper.csv", comment="#")
    #df_inf["mx"]
    #x1,y1 = df_inf["mx"],df_inf["xs"]
    
    #df_sup = pd.read_csv("DAMICS_sup.csv", comment="#")
    #x2,y2 = df_sup["mx"],df_sup["xs"]

    #plt.plot(x1,y1, 'o',label="DAMICM Upper Limit Papers")
    #plt.fill(
    #    np.append(x1, x2[::-1]),
    #    np.append(y1, y2[::-1]), color="r", alpha = 0.6, label="DAMIC Paper"
    #)

    xs_mean = np.median(xs_0, axis=1)
    xs_std = np.std(xs_0, axis=1)
    plt.plot(mass_x*1e-6,xs_mean,'--', color = "r")
    plt.fill_between(mass_x*1e-6,xs_mean-xs_std,(xs_mean+xs_std), label=r"q=0", alpha = 0.4, color = "r")

    xs_mean = np.median(xs_1, axis=1)
    xs_std = np.std(xs_1, axis=1)
    plt.plot(mass_x*1e-6,xs_mean,'--', color = "b")
    plt.fill_between(mass_x*1e-6,xs_mean-xs_std,(xs_mean+xs_std), label=r"q=1", alpha = 0.4, color = "b")

 
    xs_mean = np.median(xs_2, axis=1)
    xs_std = np.std(xs_2, axis=1)
    plt.plot(mass_x*1e-6,xs_mean,'--', color = "k")
    plt.fill_between(mass_x*1e-6,xs_mean-xs_std,(xs_mean+xs_std), label=r"q=2", alpha = 0.4, color = "k")

 
    
    plt.xlabel(r"Dark Matter Mass [MeV]")
    plt.ylabel(r"Dark Matter-e$^-$ $\sigma$ [cm$^2$]")
    plt.title("DAMIC-SNOLAB Upper Limits on Cross Section")
    plt.yscale("log")
    plt.xscale("log")
    plt.ylim(1e-36,1e-27)
    plt.xlim(0.5,100)
    #plt.xticks([1,5,10], [1,5,10])
    #plt.yticks([10**i for i in range(-44,-34)], [r"$10^{"+str(i)+"}$" for i in range(-44,-34)])
    plt.grid(True, 'both')
    plt.legend(loc="best")
    plt.show()

Dark Current [e/pix_binned*days]:  0.000225
Total Exposure [days]:  262.83333333333337
------Mass 600000.0 GeV-------
------------ Using Si model -----------
Number of Total Pixels:  328320000
Number of Signal events:  0
Lamb:  0.00985625
